<a href="https://colab.research.google.com/github/naokityokoyama/HDC/blob/main/HDC_vs_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unidecode num2words evaluate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=205feec7660613506c84c13f095d815883be6c11601295e592adaf9f52d9c288
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built docopt


In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertForSequenceClassification
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import re
import zipfile
from unidecode import unidecode
import string
from num2words import num2words

from tqdm import tqdm
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from datasets import DatasetDict, Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import evaluate

In [5]:
os.environ["WANDB_DISABLED"] = "true"

In [6]:
path_origin = '/content/drive/MyDrive/uff/fake.zip'
path_destino = '/content/'
with zipfile.ZipFile(path_origin, "r") as zip_ref:
    zip_ref.extractall(path_destino)

In [7]:
#build dataset

df_fake = pd.read_csv('/content/fakes.csv')[['text']]
df_fake['target'] = 1
df_true = pd.read_csv('/content/true.csv')[['text']]
df_true['target'] = 0
df = pd.concat([df_fake, df_true]).reset_index(drop=True)

In [24]:
# Definir o tamanho da amostra
sample_size = 20000

# Criar uma amostra balanceada
df = df.groupby("target", group_keys=False).apply(lambda x: resample(x, n_samples=sample_size // df["target"].nunique(), random_state=42))
df = df.reset_index(drop=True)

/tmp/ipython-input-1541948993.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("target", group_keys=False).apply(lambda x: resample(x, n_samples=sample_size // df["target"].nunique(), random_state=42))


In [25]:
#clean

def n2w(texto:str)->str:
  padrao = r"\d+"
  numeros = re.findall(padrao, texto)
  for numero in numeros:
    extenso = num2words(numero, lang='pt')
    texto = texto.replace(numero, extenso)
  return texto

In [26]:
for repet in tqdm(range(2)):  #bug para rodar 2x
  df['text'] = df['text'].str.lower()
  df['text'] = df['text'].str.replace(f"[{string.punctuation}]", "", regex=True)
  df['text'] = df['text'].apply(lambda x: ' '.join(x.split()))
  df['text'] = df['text'].str.replace('"', '').str.replace('\\', '')
  df['text'] = df['text'].apply(n2w)
  df['text'] = df['text'].apply(unidecode)

100%|██████████| 2/2 [00:01<00:00,  1.47it/s]


In [27]:
#create X and y
X = df['text']
y = df['target']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [29]:
train = Dataset.from_dict({'text': X_train, 'labels': y_train})
val = Dataset.from_dict({'text': X_val, 'labels': y_val})
test = Dataset.from_dict({'text': X_test, 'labels': y_test})
dataset_dict = DatasetDict({'train': train, 'test': test})
dataset_dict = DatasetDict({'train': train, 'validation': val, 'test': test})
num_labels = len(set(np.array(y)))

In [30]:
# define pre-trained model path
model_path = "google-bert/bert-base-uncased"

# load model tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [31]:
model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                           num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
# freeze all base model parameters
for name, param in model.base_model.named_parameters():
    param.requires_grad = False

# unfreeze base model pooling layers
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

In [33]:
# define text preprocessing
def preprocess_function(examples):
    # return tokenized text with truncation
    return tokenizer(examples["text"], truncation=True)

# preprocess all datasets
tokenized_data = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [34]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [35]:
# load metrics
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    # get predictions
    predictions, labels = eval_pred

    # apply softmax to get probabilities
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1,
                                                                 keepdims=True)
    # use probabilities of the positive class for ROC AUC
    positive_class_probs = probabilities[:, 1]
    # compute auc
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs,
                                     references=labels)['roc_auc'],3)

    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)
    # compute accuracy
    acc = np.round(accuracy.compute(predictions=predicted_classes,
                                     references=labels)['accuracy'],3)

    return {"Accuracy": acc, "AUC": auc}

In [36]:
# hyperparameters
lr = 2e-4
batch_size = 32
num_epochs = 2

training_args = TrainingArguments(
    output_dir="bert-phishing-classifier_teacher",
    report_to="none",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [37]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/tmp/ipython-input-2732273287.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy,Auc
1,0.660200,0.622757,0.658000,0.713000
2,0.637900,0.615155,0.663000,0.727000


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


TrainOutput(global_step=750, training_loss=0.6490735066731771, metrics={'train_runtime': 231.5187, 'train_samples_per_second': 103.663, 'train_steps_per_second': 3.239, 'total_flos': 1504797903367680.0, 'train_loss': 0.6490735066731771, 'epoch': 2.0})

In [38]:
# apply model to validation dataset
predictions = trainer.predict(tokenized_data["validation"])

# Extract the logits and labels from the predictions object
logits = predictions.predictions
labels = predictions.label_ids

# Use your compute_metrics function
metrics = compute_metrics((logits, labels))
print(metrics)



/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'Accuracy': np.float64(0.682), 'AUC': np.float64(0.74)}
